# 📢 MedReach
**AI-Powered HCP Email Campaign System**

---

🧭 **System Overview**

**MedReach** is an AI-powered multi-agent orchestration platform that automates personalized email campaigns to healthcare providers (HCPs) by intelligently selecting optimal messaging strategies and executing end-to-end campaign delivery.


📝 **User Request:** "Send a campaign about GlucoMax to endocrinologists in the Northeast regions"


🗂️ **Input Data**:

```
doctors.csv
├── doctor_id: Unique identifier
├── name: Healthcare provider full name
├── email: Professional email address
├── specialty: Medical specialty/practice area
└── region: Geographic location
```

```
doctors.csv
-----------
doctor_id,name,email,specialty,region
1,Dr. Smith,smith@hospital.com,Endocrinology,Northeast
2,Dr. Jones,jones@clinic.com,Cardiology,Southeast
3,Dr. Brown,brown@medical.com,Endocrinology,West
```

🧩 **Multi-Agent Workflow Architecture**

![](https://github.com/lisekarimi/agentverse/blob/main/assets/05_medreach-workflow.png?raw=true)



📤 **Output Data**

📊 **Campaign Tracking**

```
sent_emails.csv
├── campaign_id: Unique campaign identifier
├── doctor_id: Recipient identifier
├── doctor_name: Recipient full name
├── email_variant: Selected messaging strategy
├── sent_at: Timestamp (ISO 8601)
├── delivery_status: Success/failure indicator
└── tracking_metadata: Additional performance metrics
```

```
sent_emails.csv
---------------
campaign_id,doctor_id,doctor_name,email_variant,sent_at,status
camp_001,1,Dr. Smith,roi_focused,2025-10-30 10:15:23,sent
camp_001,3,Dr. Brown,roi_focused,2025-10-30 10:15:24,sent
```

🏗️ **Technical Architecture**

- Agent Hierarchy
    - **Orchestration Layer**: Campaign Manager Agent
    - **Specialization Layer**: Content Writer Agents (Evidence/ROI/Safety)
    - **Execution Layer**: Email Distribution Agent

- Integration Points
    - **Data Layer**: CSV file system
    - **Messaging API**: Resend email service
    - **AI Engine**: OpenAI GPT-4 model family

- Design Patterns
    - **Function Tools**: Direct Python code execution for data operations and API integrations
    - **Agent-as-Tool**: Content writers return control to manager
    - **Agent Handoff**: Distribution agent assumes full control post-selection

---

- 🌍 Task: AI-powered multi-agent system that automates personalized email campaigns to healthcare providers
- 🧠 Model: GPT-4o-mini, Gemini 2.5 Flash, Llama 3.3 70B
- 🎯 Process: Data Loading → Audience Filtering → Content Generation → Email Distribution
- 📌 Output Format: Email, CSV tracking file with campaign metadata and sent email logs
- 🔧 Tools: Resend API, CSV file operations, LLMs
- 🧑‍💻 Skill Level: Intermediate

🛠️ Requirements
- ⚙️ Hardware: ✅ CPU is sufficient
- External Services Required:
    - Resend API (https://resend.com/) - Email delivery service for sending campaign emails
    - OpenAI Platform (https://platform.openai.com/logs?api=traces) - For monitoring and tracking agent execution traces
- API keys:
    - 🔑 OpenAI API Key
    - 🔑 OpenRouter API Key
    - 🔑 Groq API Key
    - 🔑 Resend API Key
- 📧 Email address for testing
- IPython environment (Jupyter/Colab)

---
📢 Discover more Agentic AI notebooks on my [GitHub repository](https://github.com/lisekarimi/agentverse) and explore additional AI projects on my [portfolio](https://lisekarimi.com).


## 1. 🧰Imports and Environment Setup

**Note:** You'll need to create a `.env` file in your project root with:

```
OPENAI_API_KEY=
GROQ_API_KEY=
OPENROUTER_API_KEY=
RESEND_API_KEY=
TO_EMAIL=your_email
``` 



In [ ]:
import os
import pandas as pd
import json
import requests
from pydantic import BaseModel
from typing import List, Optional
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from openai import AsyncOpenAI

In [ ]:
# Load environment variables
load_dotenv(override=True)

#LLM
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

print(f"   Groq API: {'✓ Configured' if groq_api_key else '✗ Missing'}")
print(f"   OpenRouter API: {'✓ Configured' if openrouter_api_key else '✗ Missing'}")

# Setup Resend API key
RESEND_API_KEY = os.getenv("RESEND_API_KEY")
TO_EMAIL = os.getenv("TO_EMAIL")
NAME = os.getenv("NAME")

print(f"   Resend API: {'✓ Configured' if RESEND_API_KEY else '✗ Missing'}")
print(f"   To Email: {'✓ Configured' if TO_EMAIL else '✗ Missing'}")
print(f"   Name: {'✓ Configured' if NAME else '✗ Missing'}")

In [ ]:
# Constants
file_path_input = 'data/input/doctors.csv'
FROM_EMAIL= "onboarding@resend.dev" # default email for testing

MODEL = "gpt-4o-mini"
GROQ_MODEL = "llama-3.3-70b-versatile"
OPENROUTER_MODEL = "google/gemini-2.5-flash"

In [ ]:
# LLM
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
openrouter_client = AsyncOpenAI(base_url=OPENROUTER_BASE_URL, api_key=openrouter_api_key)

llama3_3_model = OpenAIChatCompletionsModel(model=GROQ_MODEL, openai_client=groq_client)
gemini_model = OpenAIChatCompletionsModel(model=OPENROUTER_MODEL, openai_client=openrouter_client)

In [ ]:
# Simple test - just one line
response = await openrouter_client.chat.completions.create(model=OPENROUTER_MODEL, messages=[{"role": "user", "content": "Say hello"}])
print(response.choices[0].message.content)

## 2. 🏭Create Fake Data

In [ ]:
# ======================================
# FAKE DATA FOR LEARNING PURPOSES ONLY
# ======================================
# This is sample data to demonstrate multi-agent systems.
# In production, this would connect to a real HCP database.

os.makedirs('data/input', exist_ok=True)
os.makedirs('data/output', exist_ok=True)

doctors_data = {
    'doctor_id': [1, 2, 3, 4],
    'name': [
        'Dr. Sarah Johnson',
        'Dr. Michael Chen',
        NAME,
        'Dr. James Williams'
    ],
    'email': [
        'fake1@example.com',
        'fake2@example.com',
        TO_EMAIL,
        'fake3@example.com'
    ],
    'specialty': [
        'Endocrinology',
        'Cardiology',
        'Endocrinology',  # Will be selected ✅
        'Orthopedics'
    ],
    'region': [
        'Midwest',
        'West',
        'Northeast',  # Will be selected ✅
        'Midwest'
    ]
}

# Create DataFrame
df = pd.DataFrame(doctors_data)

# Save to CSV
df.to_csv('data/input/doctors.csv', index=False)

# Display results
print("✅ doctors.csv created successfully!")
print(f"   Total doctors: {len(df)}")
print("\n⚠️  Note: Using fake data for educational demonstration")
print("\n📋 Full Dataset:")
df.head()

## 3. 🛠️Function Tools: Data Operations

In [ ]:
# Define the Doctor model
class Doctor(BaseModel):
    doctor_id: int
    name: str
    email: str
    specialty: str
    region: str

In [ ]:
@function_tool
def read_csv(file_path_input: str):
    """Read doctor data from CSV file"""
    df = pd.read_csv(file_path_input)
    print(f"✅ Loaded {len(df)} doctors from {file_path_input}")
    return df.to_dict('records')  # Return as list of dictionaries

In [ ]:
@function_tool
def filter_audience(doctors_list: List[Doctor], specialty: Optional[str] = None, region: Optional[str] = None):
    """Filter doctors by specialty and/or region"""
    # Convert Pydantic models to dicts for DataFrame
    doctors_dicts = [doc.model_dump() for doc in doctors_list]
    df = pd.DataFrame(doctors_dicts)

    if specialty:
        df = df[df['specialty'] == specialty]
    if region:
        df = df[df['region'] == region]

    print(f"✅ Filtered to {len(df)} doctors")
    if specialty:
        print(f"   Specialty: {specialty}")
    if region:
        print(f"   Region: {region}")

    print("\n📧 Emailing to:")
    for email in df['email'].tolist():
        print(f"   • {email}")

    return df.to_dict('records')

In [ ]:
# ============================================
# TEST THE FUNCTIONS
# ============================================
# Note: To test these functions directly, you must COMMENT OUT the @function_tool decorator
# because @function_tool wraps functions for agent use only - they cannot be called directly by us.
# After testing, UNCOMMENT @function_tool so agents can use them as tools.

# print("📋 Testing data functions...\n")
# doctors = read_csv('data/input/doctors.csv')
# endocrinologists = filter_audience(doctors, specialty='Endocrinology', region='Northeast')

# print("\nFiltered result:")
# for doc in endocrinologists:
#     print(f"   • {doc['name']} - {doc['email']}")

## 4. ✍️Create Writer Agents (Agent-as-Tool Pattern)

In [ ]:
# Shared formatting instructions for all writers
WRITER_FORMAT_INSTRUCTIONS = f"""
IMPORTANT FORMAT:
- Start with a professional greeting (e.g., "Dear Dr. [Last Name],")
- End with signature block:
  Best regards,
  GlucoMax Medical Affairs Team
  Medical Science Liaison
  GlucoMax Pharmaceuticals
  {FROM_EMAIL} | +1-800-GLUCOMAX

DO NOT add copyright notices, legal disclaimers, or footer text.
Write complete, ready-to-send emails.
"""

In [ ]:
# 1. Evidence-Based Writer Agent
evidence_instructions = f"""
You are a medical writer for GlucoMax Pharmaceuticals writing professional sales emails about GlucoMax, our diabetes medication.
Focus on clinical evidence and scientific data.
Emphasize clinical trial results, efficacy rates, and peer-reviewed research.
Keep the tone professional and data-driven.

{WRITER_FORMAT_INSTRUCTIONS}
"""

evidence_writer_agent = Agent(
    name="Evidence-Based Writer",
    instructions=evidence_instructions,
    model=MODEL
)
evidence_writer_agent

In [ ]:
# 2. ROI-Focused Writer Agent
roi_instructions = f"""
You are a medical writer for GlucoMax Pharmaceuticals writing sales emails about GlucoMax, our diabetes medication.
Focus on cost-effectiveness and return on investment.
Emphasize cost savings, patient outcomes that reduce healthcare spending, and economic benefits.
Appeal to budget-conscious decision makers.

{WRITER_FORMAT_INSTRUCTIONS}
"""

roi_writer_agent = Agent(
    name="ROI-Focused Writer",
    instructions=roi_instructions,
    model=gemini_model
    # model=MODEL
)

In [ ]:
# 3. Safety Profile Writer Agent
safety_instructions = f"""
You are a medical writer for GlucoMax Pharmaceuticals writing sales emails about GlucoMax, our diabetes medication.
Focus on drug safety and tolerability.
Emphasize low side effect profiles, long-term safety data, and patient comfort.
Address common safety concerns proactively.

{WRITER_FORMAT_INSTRUCTIONS}
"""

safety_writer_agent = Agent(
    name="Safety Profile Writer",
    instructions=safety_instructions,
    model=llama3_3_model
    # model=MODEL
)

In [ ]:
# Convert agents to tools
evidence_tool = evidence_writer_agent.as_tool(
    tool_name="evidence_writer",
    tool_description="Write clinical evidence-focused sales email for healthcare providers"
)

roi_tool = roi_writer_agent.as_tool(
    tool_name="roi_writer",
    tool_description="Write cost-effectiveness focused sales email emphasizing ROI"
)

safety_tool = safety_writer_agent.as_tool(
    tool_name="safety_writer",
    tool_description="Write safety and tolerability focused sales email"
)

# Gather all writer agent tools
agent_writer_tools = [evidence_tool, roi_tool, safety_tool]

agent_writer_tools

🤔 **Why Convert an Agent to a Tool?**

- **The Problem:** The **Campaign Manager agent** needs a way to "call" or "use" the writer agents. But agents can't directly talk to other agents - they can only use **tools**.

- **The Solution:**
We wrap each writer agent inside a tool so the manager can use them like any other tool.

---

📞 **Think of it Like a Phone System:**

**Without `.as_tool()`:**
- Manager: "I need to talk to the Evidence Writer"
- System: "Sorry, you can only use tools, not talk to agents directly"
- Manager: "But... I need their help!"

**With `.as_tool()`:**
- Manager: "I'll use the evidence_writer tool"
- Tool: *calls the Evidence Writer Agent internally*
- Evidence Writer Agent: *writes the email*
- Tool: *returns the email to Manager*
- Manager: "Perfect! Got the email."

---

🔧 **Technical Reason:**

The `Agent` class has a `tools` parameter that only accepts **FunctionTool** objects:

```python
campaign_manager = Agent(
    name="Manager",
    tools=[evidence_tool, roi_tool]  # ← Must be FunctionTools, not Agents!
)
```

So we use `.as_tool()` to convert:
- **Agent** → **FunctionTool** (that wraps the agent)

---

💡 **Summary:**

- Agents can only use **tools** (not other agents directly)
- `.as_tool()` wraps an agent so it becomes usable as a tool
- The manager calls the tool → tool calls the agent → agent does work → returns result

In [ ]:
# Combine function tools and agent tools
all_tools = [read_csv, filter_audience] + agent_writer_tools
all_tools

## 5. 📨Email Formatting Agents

In [ ]:
subject_instructions = """
You write compelling subject lines for cold sales emails to healthcare providers.
Create subjects that are professional, clear, and likely to get opened.
Keep them concise (under 60 characters) and relevant to the email content.
"""

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(
    name="Email Subject Writer",
    instructions=subject_instructions,
    model=MODEL
)

html_converter = Agent(
    name="HTML Email Converter",
    instructions=html_instructions,
    model=MODEL
)

# Convert to tools
subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Write a compelling subject line for a sales email"
)

html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert text/markdown email body to professional HTML format"
)

In [ ]:
@function_tool
def send_email(recipient_email: str, recipient_name: str, subject: str, html_body: str):
    """Send an email to a healthcare provider via Resend API"""

    from_email = FROM_EMAIL

    # Check for placeholders BEFORE sending
    if '[' in html_body or ']' in html_body:
        print("❌ ERROR: Email contains placeholders - not sending!")
        return {"status": "failure", "message": "Email contains unreplaced placeholders"}

    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }

    # Send email
    payload = {
        "from": f"GlucoMax Team <{from_email}>",
        "to": [recipient_email],
        "subject": subject,
        "html": html_body
    }

    response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)

    print(f"📧 Sending email to {recipient_name} ({recipient_email})...")

    if response.status_code == 200:
        return {"status": "success", "message": f"Email sent to {recipient_name}"}
    else:
        return {"status": "failure", "message": response.text}

In [ ]:
@function_tool
def log_to_csv(campaign_id: str, doctor_id: int, doctor_name: str, email_variant: str, status: str):
    """Log sent email to CSV for tracking"""

    import datetime

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    file_path_output = f'data/output/sent_emails_{timestamp}.csv'

    log_data = {
        'campaign_id': [campaign_id],
        'doctor_id': [doctor_id],
        'doctor_name': [doctor_name],
        'email_variant': [email_variant],
        'sent_at': [datetime.datetime.now().isoformat()],
        'status': [status]
    }

    df = pd.DataFrame(log_data)
    df.to_csv(file_path_output, index=False)

    print(f"📝 Logging campaign data for {doctor_name} with status: {status}")

    return {"status": "logged", "message": f"Campaign data logged to {file_path_output}"}

## 6. 📬Create Email Sender Agent

In [ ]:
email_sender_instructions = """
You are an email formatter and sender. You receive the body of an email to be sent.

Step 1: Use the html_converter tool to convert the body to HTML and save the result.
Step 2: Use the subject_writer tool to write a subject for the email and save the result.
Step 3: Use the send_email tool with the HTML body from Step 1 and the subject from Step 2.

IMPORTANT: Always pass the HTML output from html_converter to send_email, never the original text.
"""

# Email sender tools
email_sender_tools = [html_tool, subject_tool, send_email, log_to_csv]

email_sender_agent = Agent(
    name="Email Sender",
    instructions=email_sender_instructions,
    model=MODEL,
    tools=email_sender_tools,
    handoff_description="Format, send, and track emails to healthcare providers"
)

print("✅ Email Sender Agent created")
print(email_sender_agent)

In [ ]:
agent_dict = {
    "name": email_sender_agent.name,
    "model": email_sender_agent.model,
    "handoff_description": email_sender_agent.handoff_description,
    "handoffs": [handoff.name if hasattr(handoff, 'name') else str(handoff) for handoff in email_sender_agent.handoffs],
    "instructions": email_sender_agent.instructions,
    "tools": [
        {
            "name": tool.name,
            "description": tool.description,
            # "params_json_schema": tool.params_json_schema
        }
        for tool in email_sender_agent.tools
    ]
}

print("✅ Email Sender Agent created\n")
print(json.dumps(agent_dict, indent=2))

## 7. 🛡️ Create Input Guardrail - Audience Validation

In [ ]:
class CampaignRequestValidation(BaseModel):
    has_specialty: bool
    has_region: bool
    is_valid_request: bool
    missing_fields: List[str]

request_validation_agent = Agent(
    name="Campaign Request Validator",
    instructions="Check if the user's campaign request includes both a medical specialty and a geographic region. Both are required for valid campaigns.",
    output_type=CampaignRequestValidation,
    model=MODEL
)

@input_guardrail
async def validate_campaign_request(ctx, agent, message):
    result = await Runner.run(request_validation_agent, message, context=ctx.context)
    is_valid = result.final_output.is_valid_request
    return GuardrailFunctionOutput(
        output_info={"validation": result.final_output},
        tripwire_triggered=not is_valid  # Block if invalid request
    )

## 8. 🧠Create the Campaign Manager Agent (Orchestrator)

In [ ]:
campaign_manager_instructions = f"""
You are a pharmaceutical marketing campaign manager for GlucoMax, a new diabetes medication.

You use the tools given to you to create targeted email campaigns for healthcare providers.
You never generate emails yourself; you always use the writer tools.

First, use read_csv to load doctors from '{file_path_input}'.
Then use filter_audience to select doctors matching the user's criteria (specialty and region).
You try all 3 writer agent tools to generate different email versions.
You evaluate all 3 emails and explain your reasoning for which is most effective.
You pick the single best email (and only the best email).
After picking the best email, you handoff to the Email Sender Agent to format and send that one email.
"""

campaign_manager = Agent(
    name="Campaign Manager",
    instructions=campaign_manager_instructions,
    model=MODEL,
    tools=all_tools,
    handoffs=[email_sender_agent],
    input_guardrails=[validate_campaign_request]
)

agent_dict = {
    "name": campaign_manager.name,
    "model": campaign_manager.model,
    "handoff_description": campaign_manager.handoff_description,
    "handoffs": [handoff.name if hasattr(handoff, 'name') else str(handoff) for handoff in campaign_manager.handoffs],
    "instructions": campaign_manager.instructions,
    "tools": [
        {
            "name": tool.name,
            "description": tool.description,
            # "params_json_schema": tool.params_json_schema
        }
        for tool in campaign_manager.tools
    ]
}

print("✅ Campaign Manager Agent created\n")
print(json.dumps(agent_dict, indent=2))

## 9. 🚀Run the Multi-Agent Campaign System

In [ ]:
# User request
message = "Send a campaign about GlucoMax to endocrinologists in the Northeast region"

print("Starting multi-agent campaign execution...\n")

# Run with trace for visibility
with trace("MedReach Campaign With Guardrails"):
    result = await Runner.run(campaign_manager, message)

print("\n" + "=" * 60)
print("✅ Campaign execution completed!")

## 💡 Bonus: Function vs Agent Tools

🎯 **When to Use Function Tools vs Agent Tools:**

 **Function Tools** (like `read_csv`, `filter_audience`)

✅ Use when the task is **deterministic** and doesn't need AI reasoning:
- Reading a file
- Filtering data
- Sending an email
- Querying a database
- Mathematical calculations

**Why?** These are simple operations - just execute Python code. No need for expensive AI calls!

---

**Agent Tools** (like writer agents)

✅ Use when the task needs **creativity, reasoning, or language generation**:
- Writing emails
- Analyzing complex data
- Making strategic decisions
- Creating content

**Why?** These require AI intelligence - can't just write code for "write a persuasive email"!

---

💰 **Cost & Speed:**

```python
# Function tool - FREE & FAST
read_csv('data.csv')  # Just Python, instant, $0

# Agent tool - COSTS MONEY & SLOWER
evidence_writer.write_email()  # AI API call, 2-3 seconds, costs tokens
```

---

📋 **Rule of Thumb:**

- **Can you write simple Python code for it?** → Function Tool
- **Does it need AI to think/create?** → Agent Tool

That's why we use function tools for data operations but agent tools for writing! 🎯